## Unidade 01 - Projeto 01

- **Aluno**: Ailson Forte dos Santos
- **Matrícula**: 2016041269

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
arquivo = 'imd_notas.xlsx'
excel = pd.ExcelFile(arquivo)
dados = excel.parse(0)
dados.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,enen-nota
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,618.0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,618.0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,615.0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,600.0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,673.0


#### 1)  Quantas vezes um aluno precisa fazer uma determinada disicplina para ter aprovação?
    a) Por disciplina?
    b) Geral (considerando as 7  disciplinas)?

In [32]:
disc_uni = dados['disciplina_ID'].unique()
# disc_duo = {}
# for d in disc_uni:
#     disc_duo[d] = [d0 for d0 in dados['disciplina_ID'] if d0==d]
# disc_duo
dados[~dados['a_ID'].duplicated()]['disciplina_ID'].value_counts()[0:5].sum()

894

#### 2. Quais as regiões tem mais índices de aprovação nos componentes curriculares? Hist.

#### 3. Como estão caracterizados a  estatística de notas das diversas disciplinas que compoe o dataset? Boxplot
    3.a. Período (ano)
    3.b. Geral

#### 4. Existe correlação entre as notas dos diferentes componentes curriculares? Dispersão

#### 5. Existe uma relação entre o  número médio de disciplinas cursadas por aluno e  o  seu desempenho? Dispersão

#### 6. Existe correlação entre a  nota do enen e  o  status do aluno? Hist.

#### 7. Existe algum componente curricular na base de dados que possui uma alta correlação com os dados do ENEN? Disperção